# Create test dataset 
This notebook uses functions from the data_loaders and selectors modules to construct a small dataset for testing. The app GUI is avoided here so that options can be hard-coded in and the notebook can be run in the style of a python script (i.e. no need to interactively select options). <br><br>
Just to note, depending on the number of variables you use to construct your dataset and the geographic subset you want, this notebook can take a while to run.

In [1]:
%%capture

# Install notebook dependencies 
import xarray as xr 
import os 
import sys
import hvplot.xarray # For plotting
import cartopy.crs as ccrs # For plotting
import geopandas as gpd # For plotting
from shapely.geometry import Point # For plotting
import hvplot.pandas # For plotting
!pip install regionmask # Not part of pangeo docker image but a dependency of climakitae

# Install climakitae modules from local repo (not from GitHub package)
# If you've already downloaded the package from GitHub (i.e. !pip install climakitae) 
sys.path.insert(1, os.path.join(sys.path[0], '..')) # Add parent directory as path 
from climakitae.data_loaders import _read_from_catalog
from climakitae.selectors import DataSelector, LocSelectorArea

## Choose the settings for the test dataset
These settings will indicate what data you want to read in from the AWS catalog. 
The variables will be read in individually using the _read_from_catalog function, which returns an xarray DataArray. At the end, we will combine the individual DataArrays to form a single xarray Dataset object.

Valid variable options: 
RAINC, RAINNC, SWDDIF, TSK, PSFC, T2, Q2, U10, V10, SNOWNC, LWDNB, LWDNBC, LWUPB, LWUPBC, SWDNB, SWDNBC, SWUPB, SWUPBC

In [2]:
# Variables you want to read in and include in test dataset (list of strings) 
# If you just want one variable, it must be stored in a 1D list, i.e. ["T2"] 
variable_list = ["RAINC", "RAINNC", "SWDDIF", "TSK", "PSFC", "T2", "Q2", "U10", "V10", "SNOWNC", "LWDNB", "LWDNBC", "LWUPB", "LWUPBC", "SWDNB", "SWDNBC", "SWUPB", "SWUPBC"] 

# Starting year for time slice (integer)
year_start = 2022

# Ending year for time slice (integer)
year_end = 2022

# Timescale (string): hourly, daily, or monthly
timescale = "monthly" 

# Resolution (string): 3 km, 9 km , or 45 km 
resolution = "45 km" 

# Append historical data? (boolean) year_start must be < 2015
append_historial = False

## Read in the data from the AWS catalog

In [3]:
def read_data_for_var(variable, year_start=2013, year_end=2016, append_historical=True, timescale="monthly", resolution="45 km"): 
    """ Read data from catalog for a given variable
    Automates data reading so you don't have to use panel GUI (i.e. this is all hard-coded in) 
    
    """
    selections = DataSelector(append_historical=append_historical, 
                              area_average=False, 
                              name='DataSelector00101', 
                              resolution=resolution, 
                              scenario=['SSP 2-4.5 -- Middle of the Road', 'SSP 3-7.0 -- Business as Usual', 'SSP 5-8.5 -- Burn it All'], 
                              time_slice=(year_start, year_end), 
                              timescale=timescale, 
                              variable=variable)
    location = LocSelectorArea(area_subset='none', 
                               cached_area='CA', 
                               latitude=(32.5, 42), 
                               longitude=(-125.5, -114), 
                               name='LocSelectorArea00102')
    xr_da = _read_from_catalog(selections=selections, location=location)
    return xr_da

In [4]:
def progressBar(i, tot): 
    """Display a progress bar inside a for loop 
    Based on Stack Overflow answer from Mark Rushakoff: https://stackoverflow.com/questions/3002085/how-to-print-out-status-bar-and-percentage
    
    Args: 
        i (int): iteration number
        tot (int): total number of iterations
    """
    j = (i+1) / tot
    sys.stdout.write('\r[%-20s] %d%% complete' % ('='*int(20*j), 100*j))
    sys.stdout.flush()  

In [5]:
%%time

# Read in each variable individually into an xr.DataArray
print ("Reading in data from AWS for {0} variables: {1}".format(len(variable_list),', '.join(map(str, variable_list))))
xr_da_list = []
for i in range(len(variable_list)): # Iterate through list of variables
    variable = variable_list[i] # Select variable from list
    xr_da = read_data_for_var(variable=variable, year_start=year_start, year_end=year_end) # Read data 
    xr_da_list.append(xr_da) # Append xr.DataArray to list 
    progressBar(i, len(variable_list)) # Display progress bar 


# Merge to form a single xr.Dataset with several data variables
xr_ds = xr.merge(xr_da_list)

# Display data 
#display(xr_ds)

Reading in data from AWS for 18 variables: RAINC, RAINNC, SWDDIF, TSK, PSFC, T2, Q2, U10, V10, SNOWNC, LWDNB, LWDNBC, LWUPB, LWUPBC, SWDNB, SWDNBC, SWUPB, SWUPBC
[====================] 100% completeCPU times: user 5min 7s, sys: 4.16 s, total: 5min 11s
Wall time: 6min 32s


## Take a subset of only part of California
To avoid having a huge dataset, we just want to use a geographic subset. 
At the time of writing this notebook (June 20, 2022), the subsetting function built into climakitae was broken. So, instead we just subset the data using xarray. To do this, we construct a simple lat/lon bounding box of any region we're interested, and filter the data from the dataset using .where()<br><br>

For example, if we want to create an equal square using San Diego as the bottom left corner and Joshua Tree National Park for the top right corner, we would construct this box: <br>

(unknown) - - - - - (Joshua Tree)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|<br>
(San Diego) - - - - - (unknown)
<br><br>

We'll translate this to our code using coordinates, where (lon_1,lat_1) are the coordinates of Joshua Tree, and (lon_0,lat_0) are the coordinates of San Diego. From these corodinates, we can find the coorindates of the top left and bottom right corners to construct a box: <br>

(lon_1,lat_0)- - - - -(lon_1,lat_1)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|<br>
(lon_0,lat_0)- - - - -(lon_0,lat_1)<br>

Note that this likely won't result in a perfect square of data depending on the grid used. 

In [6]:
# San Diego coordinates 
lon_0 = min_lon = -117.1611
lat_0 = min_lat = 32.7157

# Joshua Tree coordinates 
lon_1 = max_lon = -115.9010
lat_1 = max_lat = 33.8734

# Crop the data 
mask_lon = (xr_ds.lon >= min_lon) & (xr_ds.lon <= max_lon)
mask_lat = (xr_ds.lat >= min_lat) & (xr_ds.lat <= max_lat)
test_dataset = xr_ds.where(mask_lon & mask_lat, drop=True)
#display(test_dataset) 

In [7]:
%%time 

# Load lazy dask data  
print("Hold tight! This step takes a while but will make the subsequent steps run faster")
test_dataset = test_dataset.compute()

Hold tight! This step takes a while but will make the subsequent steps run faster
CPU times: user 1min 37s, sys: 29.7 s, total: 2min 7s
Wall time: 5min 32s


## Display geographic region
We'll just use the first variable, timestep, scenario, and simulation. This step is just to show the user the geographic region that has been chosen for the test dataset

In [8]:
# -------------- Create plot with xarray data --------------

# Plot settings
projection = ccrs.Orthographic(-118, 40) # Projection to use 
var_to_plot = variable_list[0] # Variable to plot 

# Grab just one variable for plotting purposes to show geographic subset
# Otherwise the function takes a long time and I get bored waiting for it 
plotting_subset = test_dataset[var_to_plot].isel(time=0, scenario=0, simulation=0)

# Create data plot from xr.DataArray and store in a variable 
data_plot = plotting_subset.hvplot.quadmesh('lon','lat', 
            crs=ccrs.PlateCarree(),projection=projection, cmap="viridis",
            project=True,rasterize=False, dynamic=False, 
            features=['borders','coastline','ocean']) 


# -------------- Create plot with city coords --------------

# Create geopandas data frame for city coords 
d = {'location': ['San Diego', 'Joshua Tree'], 'geometry': [Point(lon_0,lat_0), Point(lon_1,lat_1)]}
gdf = gpd.GeoDataFrame(d, crs="EPSG:4326")

# Plot using geopandas hvplot and store in a variable 
city_plot = gdf.hvplot(projection=projection, color="red", hover_cols=['location'])


# -------------- Overlay city coords plot on data plot and display final map --------------

final_plot = data_plot * city_plot # Combine plots 
try: # Add title using attributes from dataset
    final_plot.opts(title=plotting_subset.description+" ("+plotting_subset.units+")") 
except: # Skip without raising error if attributes don't exist
    pass
display(final_plot) # Display

:Overlay
   .Borders.I   :Feature   [Longitude,Latitude]
   .Coastline.I :Feature   [Longitude,Latitude]
   .Ocean.I     :Feature   [Longitude,Latitude]
   .QuadMesh.I  :QuadMesh   [lon,lat]   (RAINC)
   .Points.I    :Points   [Longitude,Latitude]   (location)

# Output the test dataset as a netcdf file 
The filename definition has been automated using the dataset settings at the top of the notebook, but you can just comment this out and set the filename to whatever you want. 

In [9]:
# Define filename and output folder
output_folder = "test_data" # Set to "" if you want to store in your current directory (where the notebook is running)
filename = "test_dataset_"+str(year_start)+"_"+str(year_end)+"_"+timescale+"_"+resolution # Leave off .nc !
filename = filename.replace(" ", "") # Remove any spaces from the string
print("Filename: {0}".format(filename))

# Output to netcdf 
filepath = output_folder+"/"+filename+".nc"
test_dataset.to_netcdf(path=filepath, mode='w') # Output 
print("File saved to: {0}".format(filepath))

Filename: test_dataset_2022_2022_monthly_45km
File saved to: test_data/test_dataset_2022_2022_monthly_45km.nc
